## Load Dataset

In [1]:
from datasets import load_dataset
import os


# Specify the dataset name and the cache directory
dataset_name = "SakanaAI/AI-CUDA-Engineer-Archive"
cache_dir = "./cache_dir"
os.environ['TORCH_HOME'] = cache_dir
os.environ['TORCH_EXTENSIONS_DIR'] = cache_dir


# Load the dataset
dataset = load_dataset(dataset_name, cache_dir=cache_dir)

# Print the dataset to verify
print(dataset)

/DISK/conda_envs/cs234/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    level_1: Dataset({
        features: ['Op_Name', 'Level_ID', 'Task_ID', 'Kernel_Name', 'CUDA_Runtime', 'PyTorch_Native_Runtime', 'PyTorch_Compile_Runtime', 'CUDA_Speedup_Native', 'CUDA_Speedup_Compile', 'CUDA_Code', 'PyTorch_Code_Module', 'PyTorch_Code_Functional', 'Correct', 'Max_Diff', 'Error', 'NCU_Profile', 'Torch_Profile', 'Clang_Tidy', '__index_level_0__'],
        num_rows: 12157
    })
    level_2: Dataset({
        features: ['Op_Name', 'Level_ID', 'Task_ID', 'Kernel_Name', 'CUDA_Runtime', 'PyTorch_Native_Runtime', 'PyTorch_Compile_Runtime', 'CUDA_Speedup_Native', 'CUDA_Speedup_Compile', 'CUDA_Code', 'PyTorch_Code_Module', 'PyTorch_Code_Functional', 'Correct', 'Max_Diff', 'Error', 'NCU_Profile', 'Torch_Profile', 'Clang_Tidy', '__index_level_0__'],
        num_rows: 12938
    })
    level_3: Dataset({
        features: ['Op_Name', 'Level_ID', 'Task_ID', 'Kernel_Name', 'CUDA_Runtime', 'PyTorch_Native_Runtime', 'PyTorch_Compile_Runtime', 'CUDA_Speedup_Native', '

In [2]:
df_l1 = dataset["level_1"].to_pandas()
l1_samples = df_l1[df_l1.Kernel_Name == df_l1.Op_Name]

## Load Model

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_q14 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
# model_q7 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
model_q1 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# model_cpu = "HuggingFaceTB/SmolLM-135M"
# model_llama = "meta-llama/Meta-Llama-3-8B"

# tokenizer = AutoTokenizer.from_pretrained(model_q1, torch_dtype="auto", load_in_8bit=True, device_map="auto", cache_dir=cache_dir)
# model = AutoModelForCausalLM.from_pretrained(model_q1, torch_dtype="auto", load_in_8bit=True, device_map="auto", cache_dir=cache_dir)

tokenizer = AutoTokenizer.from_pretrained(model_q1,torch_dtype="auto", cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained(model_q1, torch_dtype="auto", cache_dir=cache_dir)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [4]:
import gc
gc.collect()

import torch
with torch.cuda.device(0):  # explicitly set GPU 0 if needed
    torch.cuda.empty_cache()
    
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [5]:
import torch
device = torch.device('cuda:0')
model = model.to(device)
# tokenizer = tokenizer.to(device)

## Misc Testing

In [6]:
# Define the prompt
prompt = "what is the solution of x^2 - 2x + 1 = 0?<think>"

# prompt = "what is the second planet from the Sun?<think>"
# prompt = l1_samples.iloc[0]["PyTorch_Code_Functional"]

# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens=100) #max_length=10_000

# Decode the generated tokens to get the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

inputs = inputs.to('cpu')

print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


KeyboardInterrupt: 

## Prompting 

In [7]:
df_l1.iloc[0].PyTorch_Code_Functional

'import torch\nimport torch.nn as nn\nimport torch.nn.functional as F\n\n\ndef module_fn(A: torch.Tensor, B: torch.Tensor) -> torch.Tensor:\n    """\n    Performs a single square matrix multiplication (C = A * B).\n\n    Args:\n        A (torch.Tensor): Input matrix A of shape (N, N).\n        B (torch.Tensor): Input matrix B of shape (N, N).\n\n    Returns:\n        torch.Tensor: Output matrix C of shape (N, N).\n    """\n    return torch.matmul(A, B)\n\n\nclass Model(nn.Module):\n    """\n    Simple model that performs a single square matrix multiplication (C = A * B)\n    """\n\n    def __init__(self):\n        super(Model, self).__init__()\n\n    def forward(self, A: torch.Tensor, B: torch.Tensor, fn=module_fn) -> torch.Tensor:\n        return fn(A, B)\n\n\nN = 2048\n\n\ndef get_inputs():\n    A = torch.randn(N, N)\n    B = torch.randn(N, N)\n    return [A, B]\n\n\ndef get_init_inputs():\n    return []  # No special initialization inputs needed\n'

In [23]:
%load_ext autoreload
%autoreload 2
from prompting import prompt_generate_custom_cuda_from_prompt_template

In [9]:
functional_str = df_l1.iloc[0].PyTorch_Code_Functional
prompt = prompt_generate_custom_cuda_from_prompt_template(functional_str, add_think=True)

In [10]:
print(prompt)

You write custom CUDA kernels to replace the pytorch operators in the given architecture to guarantee correctness and valid compiilation, and secondarily to get speedups. 

You may replace multiple operators with custom implementations, consider operator fusion opportunities (combining multiple operators into a single kernel, for example, combining matmul+relu), or algorithmic changes (such as online softmax). You are only limited by your imagination.

Here's an example to show you the syntax of the architecture you will see implemented in torch. The example architecture is for element-wise addition: 

```
import torch
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, a, b):
        return a + b


def get_inputs():
    # randomly generate input tensors based on the model architecture
    a = torch.randn(1, 128).cuda()
    b = torch.randn(1, 128).cuda()
    return [a, b]


def

In [14]:
# Define the prompt
functional_str = df_l1.iloc[0].PyTorch_Code_Functional
prompt = prompt_generate_custom_cuda_from_prompt_template(functional_str, add_think=False)

# Tokenize the input promptb
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens=3_000)

# Decode the generated tokens to get the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [15]:
outputs.shape

torch.Size([1, 1669])

In [16]:
print(response)

You write custom CUDA kernels to replace the pytorch operators in the given architecture to guarantee correctness and valid compiilation, and secondarily to get speedups. 

You may replace multiple operators with custom implementations, consider operator fusion opportunities (combining multiple operators into a single kernel, for example, combining matmul+relu), or algorithmic changes (such as online softmax). You are only limited by your imagination.

Here's an example to show you the syntax of the architecture you will see implemented in torch. The example architecture is for element-wise addition: 

```
import torch
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, a, b):
        return a + b


def get_inputs():
    # randomly generate input tensors based on the model architecture
    a = torch.randn(1, 128).cuda()
    b = torch.randn(1, 128).cuda()
    return [a, b]


def

In [83]:
working_pytorch_functional = '''
import torch
import torch.nn as nn
import torch.nn.functional as F


def module_fn(A: torch.Tensor, B: torch.Tensor) -> torch.Tensor:
    """
    Performs a single square matrix multiplication (C = A * B).

    Args:
        A (torch.Tensor): Input matrix A of shape (N, N).
        B (torch.Tensor): Input matrix B of shape (N, N).

    Returns:
        torch.Tensor: Output matrix C of shape (N, N).
    """
    return torch.matmul(A, B)


class Model(nn.Module):
    """
    Simple model that performs a single square matrix multiplication (C = A * B)
    """

    def __init__(self):
        super(Model, self).__init__()

    def forward(self, A: torch.Tensor, B: torch.Tensor, fn=module_fn) -> torch.Tensor:
        return fn(A, B)


N = 2048


def get_inputs():
    A = torch.randn(N, N).cuda()
    B = torch.randn(N, N).cuda()
    return [A, B]


def get_init_inputs():
    return []  # No special initialization inputs needed
'''


working_cuda_source = """
#include <torch/extension.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <c10/cuda/CUDAException.h>

#define TILE_SIZE 16

#define CHECK_CUDA(x) TORCH_CHECK(x.is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)
#define CHECK_FLOAT(x) TORCH_CHECK(x.scalar_type() == torch::kFloat32, #x " must be a float32 tensor")

__global__ void matmul_tiled_kernel(const float* __restrict__ A, const float* __restrict__ B, float* __restrict__ C, int N) {
    __shared__ float As[TILE_SIZE][TILE_SIZE];
    __shared__ float Bs[TILE_SIZE][TILE_SIZE];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = blockIdx.y * TILE_SIZE + ty;
    int col = blockIdx.x * TILE_SIZE + tx;

    float C_value = 0.0f;

    for (int m = 0; m < (N + TILE_SIZE - 1) / TILE_SIZE; ++m) {
        // Load tiles into shared memory
        if (row < N && m * TILE_SIZE + tx < N)
            As[ty][tx] = A[row * N + m * TILE_SIZE + tx];
        else
            As[ty][tx] = 0.0f;

        if (col < N && m * TILE_SIZE + ty < N)
            Bs[ty][tx] = B[(m * TILE_SIZE + ty) * N + col];
        else
            Bs[ty][tx] = 0.0f;

        __syncthreads();

        // Compute partial product
        for (int k = 0; k < TILE_SIZE; ++k) {
            C_value += As[ty][k] * Bs[k][tx];
        }

        __syncthreads();
    }

    // Write the result
    if (row < N && col < N)
        C[row * N + col] = C_value;
}

torch::Tensor forward(torch::Tensor A, torch::Tensor B) {
    CHECK_INPUT(A);
    CHECK_INPUT(B);
    CHECK_FLOAT(A);
    CHECK_FLOAT(B);

    TORCH_CHECK(A.dim() == 2 && A.size(0) == A.size(1), "A must be a square matrix");
    TORCH_CHECK(B.dim() == 2 && B.size(0) == B.size(1), "B must be a square matrix");
    TORCH_CHECK(A.size(0) == B.size(0), "A and B must be of the same size");

    int64_t N = A.size(0);

    auto C = torch::zeros({N, N}, A.options());

    const float* A_data = A.data_ptr<float>();
    const float* B_data = B.data_ptr<float>();
    float* C_data = C.data_ptr<float>();

    dim3 threadsPerBlock(TILE_SIZE, TILE_SIZE);
    dim3 blocksPerGrid((N + TILE_SIZE - 1) / TILE_SIZE, (N + TILE_SIZE - 1) / TILE_SIZE);

    matmul_tiled_kernel<<<blocksPerGrid, threadsPerBlock>>>(A_data, B_data, C_data, N);

    // Check for kernel launch errors
    C10_CUDA_CHECK(cudaGetLastError());

    return C;
}
"""

working_cpp_source = (
    "torch::Tensor forward(torch::Tensor A, torch::Tensor B);"
)

working_response = '''
</think>
cuda_source = """
#include <torch/extension.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <c10/cuda/CUDAException.h>

#define TILE_SIZE 16

#define CHECK_CUDA(x) TORCH_CHECK(x.is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)
#define CHECK_FLOAT(x) TORCH_CHECK(x.scalar_type() == torch::kFloat32, #x " must be a float32 tensor")

__global__ void matmul_tiled_kernel(const float* __restrict__ A, const float* __restrict__ B, float* __restrict__ C, int N) {
    __shared__ float As[TILE_SIZE][TILE_SIZE];
    __shared__ float Bs[TILE_SIZE][TILE_SIZE];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = blockIdx.y * TILE_SIZE + ty;
    int col = blockIdx.x * TILE_SIZE + tx;

    float C_value = 0.0f;

    for (int m = 0; m < (N + TILE_SIZE - 1) / TILE_SIZE; ++m) {
        // Load tiles into shared memory
        if (row < N && m * TILE_SIZE + tx < N)
            As[ty][tx] = A[row * N + m * TILE_SIZE + tx];
        else
            As[ty][tx] = 0.0f;

        if (col < N && m * TILE_SIZE + ty < N)
            Bs[ty][tx] = B[(m * TILE_SIZE + ty) * N + col];
        else
            Bs[ty][tx] = 0.0f;

        __syncthreads();

        // Compute partial product
        for (int k = 0; k < TILE_SIZE; ++k) {
            C_value += As[ty][k] * Bs[k][tx];
        }

        __syncthreads();
    }

    // Write the result
    if (row < N && col < N)
        C[row * N + col] = C_value;
}

torch::Tensor forward(torch::Tensor A, torch::Tensor B) {
    CHECK_INPUT(A);
    CHECK_INPUT(B);
    CHECK_FLOAT(A);
    CHECK_FLOAT(B);

    TORCH_CHECK(A.dim() == 2 && A.size(0) == A.size(1), "A must be a square matrix");
    TORCH_CHECK(B.dim() == 2 && B.size(0) == B.size(1), "B must be a square matrix");
    TORCH_CHECK(A.size(0) == B.size(0), "A and B must be of the same size");

    int64_t N = A.size(0);

    auto C = torch::zeros({N, N}, A.options());

    const float* A_data = A.data_ptr<float>();
    const float* B_data = B.data_ptr<float>();
    float* C_data = C.data_ptr<float>();

    dim3 threadsPerBlock(TILE_SIZE, TILE_SIZE);
    dim3 blocksPerGrid((N + TILE_SIZE - 1) / TILE_SIZE, (N + TILE_SIZE - 1) / TILE_SIZE);

    matmul_tiled_kernel<<<blocksPerGrid, threadsPerBlock>>>(A_data, B_data, C_data, N);

    // Check for kernel launch errors
    C10_CUDA_CHECK(cudaGetLastError());

    return C;
}
"""

cpp_source = (
    "torch::Tensor forward(torch::Tensor A, torch::Tensor B);"
)
'''

In [11]:
l1_samples = df_l1[df_l1.Kernel_Name == df_l1.Op_Name]

In [84]:
from reward_model import reward

reward_val, msg = reward(l1_samples.iloc[0].PyTorch_Code_Functional, response)
print(reward_val)
print(msg)
print()
reward_val_good, msg_good = reward(working_pytorch_functional, working_response)
print(reward_val_good)
print(msg_good)

-30
The outputted CUDA kernel was not formatted correctly. Please follow the format of the examples given!



-1
Traceback (most recent call last):
  File "/DISK/conda_envs/cs234/lib/python3.11/site-packages/torch/utils/cpp_extension.py", line 2209, in _run_ninja_build
    subprocess.run(
  File "/DISK/conda_envs/cs234/lib/python3.11/subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['ninja', '-v']' returned non-zero exit status 1.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/DISK/cs234_cuda/reward_model.py", line 79, in reward
    cuda_mod = load_inline(
               ^^^^^^^^^^^^
  File "/DISK/conda_envs/cs234/lib/python3.11/site-packages/torch/utils/cpp_extension.py", line 1723, in load_inline
    return _jit_compile(
           ^^^^^^^^^^^^^
  File "/DISK/conda_envs/cs234/lib/python3.11/site-packages/torch/utils/cpp_extension.py", line 1798, in _jit_compile
    _write_ninja_file_and_build_library(
  File "/DISK/conda_envs/cs234/lib/python3.11/sit

## Sakana Eval Testing (Not Working)

In [55]:
with open(f"tasks/{df_l1.iloc[0].Op_Name}.py", "w") as f:
    f.write(df_l1.iloc[0].PyTorch_Code_Functional)
with open(f"kernels/{df_l1.iloc[0].Op_Name}.cu", "w") as f:
    f.write(df_l1.iloc[0].CUDA_Code)

File format is:

eval_kernel

task/
- torch nn module.py
- functional.py
- info.txt (dont really need this rn)

kernel/
- kernel.cu 

In [ ]:
# # Evaluation script for CUDA kernel
# # 12_Matmul_with_diagonal_matrices_
# # Evaluation script for CUDA kernel
# # 12_Matmul_with_diagonal_matrices_
# import os
# import torch
# import argparse
# from torch.utils.cpp_extension import load
# from torch.utils._pytree import tree_map
# import importlib.util
# from torch.utils.benchmark import Timer


# def easy_to_device(pytree, device):
#     return tree_map(
#         lambda x: x.to(device) if isinstance(x, torch.Tensor) else x, pytree
#     )


# def load_module_from_path(path):
#     spec = importlib.util.spec_from_file_location("module", path)
#     module = importlib.util.module_from_spec(spec)
#     spec.loader.exec_module(module)
#     return module


# def evaluate(op_name: str):
#     # parser = argparse.ArgumentParser()
#     # parser.add_argument("--op_atol", type=float, default=1e-3)
#     # parser.add_argument("--op_rtol", type=float, default=1e-1)
#     # parser.add_argument("--rep_time", type=int, default=10000)
#     # parser.add_argument("--warmup_time", type=int, default=25)
#     # args = parser.parse_args()

#     # # Get task name from info.txt
#     # with open("task/info.txt", "r") as f:
#     #     task_name = f.readline().strip()
#     #     task_name = "_".join(task_name.split("_")[1:])  # Remove problem ID

#     # Import the task module
#     # task_files = [f for f in os.listdir("tasks") if f.endswith("_functional.py")]
#     # if not task_files:
#     #     raise RuntimeError("No functional task file found")

#     task = load_module_from_path(os.path.join("tasks", op_name+'.py'))

#     # Initialize model and inputs
#     device_1 = torch.device("cuda:0")
#     torch.manual_seed(0)
#     inputs = task.get_inputs()
#     init_inputs = task.get_init_inputs()
#     model = task.Model(*init_inputs)

#     # Load CUDA kernel
#     # kernel_files = [f for f in os.listdir("kernel") if f.endswith(".cu")]
#     # if not kernel_files:
#     #     raise RuntimeError("No CUDA kernel file found")
    
#     task_name = "_".join(op_name.split("_")[1:])  # Remove problem ID
#     cuda_module = load(
#         name=task_name,
#         sources=[os.path.join("kernels", op_name+'.cu')],
#         extra_cuda_cflags=["-O3", "--use_fast_math"],
#         with_cuda=True,
#         verbose=True,
#     )

#     # Test for correctness
#     with torch.no_grad():
#         cuda_output = model.to(device_1)(
#             *easy_to_device(inputs, device_1), fn=cuda_module.forward
#         )
#         torch_output = model.to(device_1)(
#             *easy_to_device(inputs, device_1), fn=task.module_fn
#         )

#     rtol_default = 1e-1
#     atol_default = 1e-3

#     correct = torch.allclose(
#         torch_output.cpu(),
#         cuda_output.cpu(),
#         rtol=rtol_default,
#         atol=atol_default,
#     )
#     max_diff = torch.max(torch.abs(torch_output.cpu() - cuda_output.cpu())).item()
#     print(f"Tested CUDA kernel - Correct: {correct}, Max Diff: {max_diff}")

#     if correct:
#         # Evaluate CUDA kernel performance
#         cuda_timer = Timer(
#             stmt="model(*inputs, fn=cuda_module.forward)",
#             globals={
#                 "model": model.to(device_1),
#                 "inputs": easy_to_device(inputs, device_1),
#                 "cuda_module": cuda_module,
#             },
#         )
#         cuda_runtime = cuda_timer.timeit(args.rep_time).mean * 1000
#         print(f"Evaluated CUDA kernel - Runtime: {cuda_runtime:.3f} ms")

#         # Evaluate PyTorch baseline performance
#         torch_timer = Timer(
#             stmt="model(*inputs, fn=task.module_fn)",
#             globals={
#                 "model": model.to(device_1),
#                 "inputs": easy_to_device(inputs, device_1),
#                 "task": task,
#             },
#         )
#         torch_runtime = torch_timer.timeit(args.rep_time).mean * 1000
#         print(f"Evaluated PyTorch baseline - Runtime: {torch_runtime:.3f} ms")

#         # Evaluate torch compile performance
#         torch_fn = task.module_fn
#         compile_fn = torch.compile(torch_fn, mode="max-autotune")
#         torch_compile_timer = Timer(
#             stmt="model(*inputs, fn=compile_fn)",
#             globals={
#                 "model": model.to(device_1),
#                 "inputs": easy_to_device(inputs, device_1),
#                 "compile_fn": compile_fn,
#             },
#         )

#         torch_compile_runtime = torch_compile_timer.timeit(args.rep_time).mean * 1000
#         print(f"Evaluated torch compile - Runtime: {torch_compile_runtime:.3f} ms")

#         print(f"Speedup over PyTorch: {torch_runtime/cuda_runtime:.2f}x")
#         print(f"Speedup over torch compile: {torch_compile_runtime/cuda_runtime:.2f}x")

#         import json

#         # Store the speedup times as a json file
#         file_path = os.path.join(os.path.dirname('speedups'), f"{op_name}.json")
#         with open(file_path, "w") as f:
#             json.dump(
#                 {
#                     "max_diff": max_diff,
#                     "cuda_runtime": cuda_runtime,
#                     "torch_runtime": torch_runtime,
#                     "torch_compile_runtime": torch_compile_runtime,
#                     "speedup_over_pytorch": torch_runtime / cuda_runtime,
#                     "speedup_over_torch_compile": torch_compile_runtime / cuda_runtime,
#                 },
#                 f,
#             )
#         print(f"Speedup times stored in {file_path}")

In [114]:
# evaluate(df_l1.iloc[0].Op_Name)

## Example code from KernelBench. Evaluation seems to work with this setup.

In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, a, b):
        return a + b


def get_inputs():
    # randomly generate input tensors based on the model architecture
    a = torch.randn(1, 128).cuda()
    b = torch.randn(1, 128).cuda()
    return [a, b]


def get_init_inputs():
    # randomly generate tensors required for initialization based on the model architecture
    return []


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.cpp_extension import load_inline

# Define the custom CUDA kernel for element-wise addition
elementwise_add_source = """
#include <torch/extension.h>
#include <cuda_runtime.h>

__global__ void elementwise_add_kernel(const float* a, const float* b, float* out, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        out[idx] = a[idx] + b[idx];
    }
}

torch::Tensor elementwise_add_cuda(torch::Tensor a, torch::Tensor b) {
    auto size = a.numel();
    auto out = torch::zeros_like(a);

    const int block_size = 256;
    const int num_blocks = (size + block_size - 1) / block_size;

    elementwise_add_kernel<<<num_blocks, block_size>>>(a.data_ptr<float>(), b.data_ptr<float>(), out.data_ptr<float>(), size);

    return out;
}
"""

elementwise_add_cpp_source = (
    "torch::Tensor elementwise_add_cuda(torch::Tensor a, torch::Tensor b);"
)

# Compile the inline CUDA code for element-wise addition
elementwise_add = load_inline(
    name="elementwise_add",
    cpp_sources=elementwise_add_cpp_source,
    cuda_sources=elementwise_add_source,
    functions=["elementwise_add_cuda"],
    verbose=True,
    extra_cflags=[""],
    extra_ldflags=[""],
)


class ModelNew(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.elementwise_add = elementwise_add

    def forward(self, a, b):
        return self.elementwise_add.elementwise_add_cuda(a, b)
    
a, b = get_inputs()
torchm = Model()
cudam = ModelNew()
torch.allclose(torchm.forward(a,b), cudam.forward(a,b) )

Using ./cache_dir as PyTorch extensions root...
The input conditions for extension module elementwise_add have changed. Bumping to version 4 and re-building as elementwise_add_v4...
Detected CUDA files, patching ldflags
Emitting ninja build file ./cache_dir/elementwise_add/build.ninja...
/DISK/conda_envs/cs234/lib/python3.11/site-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module elementwise_add_v4...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=elementwise_add_v4 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /DISK/conda_envs/cs234/lib/python3.11/site-packages/torch/include -isystem /DISK/conda_envs/cs234/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /DISK/conda_envs/cs234/lib/python3.11/site-packages/torch/include/TH -isystem /DISK/conda_envs/cs234/lib/python3.11/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /DISK/conda_envs/cs234/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17  -c /DISK/cs234_cuda/cache_dir/elementwise_add/main.cpp -o main.o 
[2/3] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output cuda.cuda.o.d -DTORCH_EXTENSION_NAME=elementwise_add_v4 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cx

Loading extension module elementwise_add_v4...


True

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def module_fn(A: torch.Tensor, B: torch.Tensor) -> torch.Tensor:
    """
    Performs a single square matrix multiplication (C = A * B).

    Args:
        A (torch.Tensor): Input matrix A of shape (N, N).
        B (torch.Tensor): Input matrix B of shape (N, N).

    Returns:
        torch.Tensor: Output matrix C of shape (N, N).
    """
    return torch.matmul(A, B)


class Model(nn.Module):
    """
    Simple model that performs a single square matrix multiplication (C = A * B)
    """

    def __init__(self):
        super(Model, self).__init__()

    def forward(self, A: torch.Tensor, B: torch.Tensor, fn=module_fn) -> torch.Tensor:
        return fn(A, B)


N = 2048


def get_inputs():
    A = torch.randn(N, N).cuda()
    B = torch.randn(N, N).cuda()
    return [A, B]


def get_init_inputs():
    return []  # No special initialization inputs needed


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.cpp_extension import load_inline

cuda_source = """
#include <torch/extension.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <c10/cuda/CUDAException.h>

#define TILE_SIZE 16

#define CHECK_CUDA(x) TORCH_CHECK(x.is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)
#define CHECK_FLOAT(x) TORCH_CHECK(x.scalar_type() == torch::kFloat32, #x " must be a float32 tensor")

__global__ void matmul_tiled_kernel(const float* __restrict__ A, const float* __restrict__ B, float* __restrict__ C, int N) {
    __shared__ float As[TILE_SIZE][TILE_SIZE];
    __shared__ float Bs[TILE_SIZE][TILE_SIZE];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = blockIdx.y * TILE_SIZE + ty;
    int col = blockIdx.x * TILE_SIZE + tx;

    float C_value = 0.0f;

    for (int m = 0; m < (N + TILE_SIZE - 1) / TILE_SIZE; ++m) {
        // Load tiles into shared memory
        if (row < N && m * TILE_SIZE + tx < N)
            As[ty][tx] = A[row * N + m * TILE_SIZE + tx];
        else
            As[ty][tx] = 0.0f;

        if (col < N && m * TILE_SIZE + ty < N)
            Bs[ty][tx] = B[(m * TILE_SIZE + ty) * N + col];
        else
            Bs[ty][tx] = 0.0f;

        __syncthreads();

        // Compute partial product
        for (int k = 0; k < TILE_SIZE; ++k) {
            C_value += As[ty][k] * Bs[k][tx];
        }

        __syncthreads();
    }

    // Write the result
    if (row < N && col < N)
        C[row * N + col] = C_value;
}

torch::Tensor forward(torch::Tensor A, torch::Tensor B) {
    CHECK_INPUT(A);
    CHECK_INPUT(B);
    CHECK_FLOAT(A);
    CHECK_FLOAT(B);

    TORCH_CHECK(A.dim() == 2 && A.size(0) == A.size(1), "A must be a square matrix");
    TORCH_CHECK(B.dim() == 2 && B.size(0) == B.size(1), "B must be a square matrix");
    TORCH_CHECK(A.size(0) == B.size(0), "A and B must be of the same size");

    int64_t N = A.size(0);

    auto C = torch::zeros({N, N}, A.options());

    const float* A_data = A.data_ptr<float>();
    const float* B_data = B.data_ptr<float>();
    float* C_data = C.data_ptr<float>();

    dim3 threadsPerBlock(TILE_SIZE, TILE_SIZE);
    dim3 blocksPerGrid((N + TILE_SIZE - 1) / TILE_SIZE, (N + TILE_SIZE - 1) / TILE_SIZE);

    matmul_tiled_kernel<<<blocksPerGrid, threadsPerBlock>>>(A_data, B_data, C_data, N);

    // Check for kernel launch errors
    C10_CUDA_CHECK(cudaGetLastError());

    return C;
}
"""

cuda_cpp_source = (
    "torch::Tensor forward(torch::Tensor A, torch::Tensor B);"
)

# Compile the inline CUDA code 
cuda_mod = load_inline(
    name="matmul",
    cpp_sources=cuda_cpp_source,
    cuda_sources=cuda_source,
    functions=["forward"],
    verbose=True,
    extra_cflags=[""],
    extra_ldflags=[""],
)


class ModelNew(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.cuda_mod = cuda_mod

    def forward(self, a, b):
        return self.cuda_mod.forward(a, b)

# QWEN 7B GENERATED

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.cpp_extension import load_inline

# # Define the custom CUDA kernel for matrix multiplication
# matmul_kernel_source = """
# #include <torch/extension.h>
# #include <cuda_runtime.h>

# __global__ void matmul_kernel(const float* a, const float* b, float* out, int n) {
#     int row = blockIdx.y * blockDim.y + threadIdx.y;
#     int col = blockIdx.x * blockDim.x + threadIdx.x;
#     float sum = 0.0f;
#     for (int k = 0; k < n; k++) {
#         sum += a[row * n + k] * b[k * n + col];
#     }
#     out[row * n + col] = sum;
# }

# torch::Tensor matmul_cuda(torch::Tensor a, torch::Tensor b) {
#     int n = a.size(1);
#     int m = b.size(0);
#     int k = b.size(1);

#     auto out_size = m * k;
#     auto out = torch::zeros(m, k, sizeof(float));

#     const int block_size = 256;
#     const int num_blocks = (out_size + block_size - 1) / block_size;

#     matmul_kernel<<<num_blocks, block_size>>>(
#         a.data_ptr<float>(), b.data_ptr<float>(), out.data_ptr<float>(), n);

#     return out;
# }
# """

# matmul_cpp_source = (
#     "torch::Tensor matmul_cuda(torch::Tensor a, torch::Tensor b);"
# )

# # Compile the inline CUDA code for matrix multiplication
# matmul = load_inline(
#     name="matmul",
#     cpp_sources=matmul_cpp_source,
#     cuda_sources=matmul_kernel_source,
#     functions=["matmul_cuda"],
#     verbose=True,
#     extra_cflags=["-O3"],
#     extra_ldflags=[""],
# )

# class ModelNew(nn.Module):
#     def __init__(self) -> None:
#         super().__init__()
#         self.matmul = matmul

#     def forward(self, a, b):
#         return self.matmul.matmul_cuda(a, b)
    
torch_mod = Model()
cuda_mod = ModelNew()

a, b = get_inputs()
# a, b, = torch.eye(N).cuda(), torch.eye(N).cuda()
print(torch.allclose(torch_mod.forward(a, b), cuda_mod.forward(a, b), rtol=1e-1, atol=1e-3))
torch_mod.forward(a, b), cuda_mod.forward(a, b)

In [ ]:
print(l1_samples.iloc[2].CUDA_Code)

In [ ]:
exec(l1_samples.iloc[2].PyTorch_Code_Functional)
cuda_src = """
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

// CUDA kernel for batched matrix multiplication: C = A * B
// Shapes: A (batch_size, M, K), B (batch_size, K, N), C (batch_size, M, N)
__global__ void bmm_kernel(
    const float* __restrict__ A,
    const float* __restrict__ B,
    float* __restrict__ C,
    int batch_size,
    int M,
    int K,
    int N
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int total = batch_size * M * N;
    if (idx >= total) return;

    int b = idx / (M * N);
    int remainder = idx % (M * N);
    int m = remainder / N;
    int n = remainder % N;

    float val = 0.0f;
    for (int k = 0; k < K; k++) {
        val += A[b * M * K + m * K + k] * B[b * K * N + k * N + n];
    }
    C[b * M * N + m * N + n] = val;
}

torch::Tensor forward_bmm(torch::Tensor A, torch::Tensor B) {
    TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
    TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
    TORCH_CHECK(A.dim() == 3, "A must be 3D");
    TORCH_CHECK(B.dim() == 3, "B must be 3D");
    TORCH_CHECK(A.size(0) == B.size(0), "Batch sizes must match");
    TORCH_CHECK(A.size(2) == B.size(1), "Inner dimensions (K) must match");

    int batch_size = A.size(0);
    int M = A.size(1);
    int K = A.size(2);
    int N = B.size(2);

    auto options = torch::TensorOptions().dtype(A.dtype()).device(A.device());
    auto C = torch::zeros({batch_size, M, N}, options);

    int total = batch_size * M * N;
    const int threads = 256;
    int blocks = (total + threads - 1) / threads;

    bmm_kernel<<<blocks, threads>>>(
        A.data_ptr<float>(),
        B.data_ptr<float>(),
        C.data_ptr<float>(),
        batch_size, M, K, N
    );

    return C;
}
"""

cuda_cpp_source = (
    "torch::Tensor forward_bmm(torch::Tensor A, torch::Tensor B);"
)

# Compile the inline CUDA code 
cuda_mod = load_inline(
    name="bmm",
    cpp_sources=cuda_cpp_source,
    cuda_sources=cuda_src,
    functions=["forward_bmm"],
    verbose=True,
    extra_cflags=[""],
    extra_ldflags=[""],
)


class ModelNew(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.cuda_mod = cuda_mod

    def forward(self, a, b):
        return self.cuda_mod.forward_bmm(a, b)

In [ ]:
a, b = get_inputs()
a = a.cuda()
b = b.cuda()

torchm = Model()
cudam = Model()
print(torch.allclose(torchm.forward(a, b), cudam.forward(a, b), rtol=1e-1, atol=1e-3))
torchm.forward(a, b), cudam.forward(a, b)

## TODOs
- ~~evaluation stuff~~
- prompting qwen for good outputs 
- ~~KernelBench method - NA~~
- coding the RL portion

## RL portion
Below implements a GRPO like method (calculating advantage from reward - mean reward in batch)

In [7]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
lora_config = LoraConfig(
    r=8,                      # rank of the low-rank matrices
    lora_alpha=16,            # scaling factor
    target_modules=["q_proj", "v_proj"],  # adjust based on your model architecture
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
# Wrap the model with LoRA. This freezes the base model parameters and injects trainable adapters.
model = get_peft_model(model, lora_config)

In [8]:
model.print_trainable_parameters()

trainable params: 1,089,536 || all params: 1,778,177,536 || trainable%: 0.0613


In [9]:
import torch
import torch.nn.functional as F
from torch.optim import AdamW, SGD, Adafactor
import torch.optim
from prompting import prompt_generate_custom_cuda_from_prompt_template, prompt_generate_reprompt
from reward_model import reward
import gc
from functools import partial
import concurrent.futures

# the batch size hyperparameters
batch_size = 2          # size of our batch (number of prompts)
reprompts = 2           # number of times we try while including the error message. includes the first prompt

# PPO/training hyperparameters
clip_range = 0.2        # clipping range for PPO
ppo_epochs = 10          # number of PPO updates per batch
num_iterations = 1      # total training iterations
log_prob_min_ratio = -10
log_prob_max_ratio = 5
lr = 1e-5

# text generation hyperparameters
temperature = 1
max_new_tokens = 3_000

# the PyTorch Code Module we're tackling.
problem_id = 0
pytorch_str = l1_samples.iloc[problem_id]['PyTorch_Code_Module']

In [10]:
# Helper function to create a mask from a list of ignore ranges.

def create_ignore_mask(seq_len, attention_mask, ignore_ranges, dtype=torch.bfloat16):
    """
    Create a 1D mask (length seq_len) where tokens in any ignore range are set to 0,
    and others are 1.
    ignore_ranges: list of tuples (start, end) with end not included.
    """
    if attention_mask is not None:
        assert len(attention_mask.shape) == 1
        mask = F.pad(attention_mask.to(dtype), (0, seq_len - attention_mask.shape[0]), mode='constant', value=1)
    else:
        mask = torch.ones(seq_len, dtype=dtype)
    for (start, end) in ignore_ranges:
        # Ensure the ignore indices are within bounds
        start = max(0, start)
        end = min(seq_len, end)
        mask[start:end] = 0.0
    return mask

In [ ]:
# optimizer = AdamW(model.parameters(), lr=lr)
# optimizer = SGD(model.parameters(), lr=lr)
optimizer = Adafactor(model.parameters(), lr=lr)
dtype = torch.bfloat16


for iteration in range(num_iterations):
    print(f'iteration num: {iteration}')
    batch_outputs = []
    batch_rewards = [0 for _ in range(batch_size)]

    ################################
    # (old policy): generate data. #
    ################################

    batch_outputs = ["" for _ in range(batch_size)]
    prompt = prompt_generate_custom_cuda_from_prompt_template(pytorch_str, add_think=False)
    prompts = tokenizer([prompt for _ in range(batch_size)]) # basically dictionary of keys 'input_ids', 'attention_mask'. in list form
    pads = [0 for _ in range(batch_size)]
    tokens_to_ignore_later = [[(0, len(ids))] for ids in prompts['input_ids']]
    # format: list of list of tuples. first index is the number of the item within the batch. 
    # for each batch, list of tuples of indices to ignore (start, end) where start is included and end is NOT.

    # variables required for the old_log_prob calculation
    gen_ids = None
    last_attention_mask = None

    for idx in range(reprompts):
        print(f'\treprompt attempt {idx}')
        inputs = tokenizer.pad(prompts, padding=True, return_tensors='pt').to(device)
        gen_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )
        if idx == reprompts - 1: # used to calculate the number of pad tokens for the last iteration
            last_attention_mask = inputs.attention_mask

        # take only the tokens after the inputs
        batch_outputs = [tokenizer.decode(ids[inputs.input_ids.shape[1]:], skip_special_tokens=True) for ids in gen_ids]
        for i, output_txt in enumerate(batch_outputs):
            print(f'\t\tcalculating reward for item {i} in batch')
            r, msg = reward(pytorch_str, output_txt)
            batch_rewards[i] += r
            reprompt = prompt_generate_reprompt(msg)

            output_ids = tokenizer(batch_outputs[i]).input_ids
            reprompt_ids = tokenizer(reprompt).input_ids

            orig_tokens = tokens_to_ignore_later[i][-1][1]
            output_tokens = len(output_ids)
            reprompt_tokens = len(reprompt_ids)

            prompts.input_ids[i].extend(output_ids)
            prompts.input_ids[i].extend(reprompt_ids)
            prompts.attention_mask[i].extend([1] * (output_tokens + reprompt_tokens))
            tokens_to_ignore_later[i].append((orig_tokens + output_tokens, orig_tokens + output_tokens + reprompt_tokens))
        
    # compute log probs. ignore any instances of pad_token using the attention mask
    with torch.no_grad():
        outputs = model(gen_ids) # this uses the last instance of gen_ids
        log_probs = F.log_softmax(outputs.logits, dim=-1)
        gen_log_probs = log_probs.gather(2, gen_ids.unsqueeze(-1)).squeeze(-1)

        # Create a masked log probability for each batch element.
        old_log_probs_list = []
        batch_size, seq_len = gen_ids.shape
        for i in range(batch_size):
            # Create a mask for this batch element using tokens_to_ignore_later.
            # tokens_to_ignore_later[i] is a list of tuples (start, end) to ignore.
            mask = create_ignore_mask(seq_len, last_attention_mask[i], tokens_to_ignore_later[i], dtype=dtype).to(gen_log_probs.device)
            masked_log_probs = gen_log_probs[i] * mask
            # Sum over the sequence to get a scalar log-prob for this example.
            old_log_prob = masked_log_probs.sum()
            old_log_probs_list.append(old_log_prob)
        
        old_log_probs_tensor = torch.stack(old_log_probs_list)
    
    print()
    print(f'rewards: {batch_rewards}')
    if batch_size > 1 and all(r == batch_rewards[0] for r in batch_rewards):
        print(f'\tthis batch had the same rewards. not performing PPO on this batch.')
        continue
    print(f'old log probs: {old_log_probs_tensor}')
    print('done with generations in old policy')
    print()

    rewards_tensor = torch.tensor(batch_rewards, dtype=dtype).to(device)

    ####################################
    # PPO Update Loop with Masking   #
    ####################################
    # In the PPO update loop, it is important that we compute the new log probs
    # over the full concatenated sequence (i.e. the updated prompts) and then apply the same mask.

    for _ in range(ppo_epochs):
        print(f'\tppo_epoch: {_}')

        # Process each batch element individually.
        for i, output_text in enumerate(batch_outputs):
            # Instead of re-tokenizing only the new output, use the entire sequence stored in prompts[i]
            full_ids = torch.tensor(prompts.input_ids[i], device=device).unsqueeze(0)
            outputs = model(full_ids)
            log_probs = F.log_softmax(outputs.logits, dim=-1)
            gen_log_probs = log_probs.gather(2, full_ids.unsqueeze(-1)).squeeze(-1)
            # Create the ignore mask using tokens_to_ignore_later for this batch item. Note None since prompts[i] shouldn't have any padding tokens, thus no attention mask necessary
            mask = create_ignore_mask(full_ids.shape[1], None, tokens_to_ignore_later[i], dtype=dtype).to(device)
            # Compute the masked sequence log probability.
            sequence_log_prob = (gen_log_probs[0] * mask).sum()

            ratio = torch.exp(torch.clamp(sequence_log_prob - old_log_probs_tensor[i], log_prob_min_ratio, log_prob_max_ratio))
            advantage = rewards_tensor[i] - rewards_tensor.mean()
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1 - clip_range, 1 + clip_range) * advantage
            loss = -torch.min(surr1, surr2).mean()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_val = loss.item()
            del full_ids, outputs, log_probs, gen_log_probs, sequence_log_prob, ratio, advantage, surr1, surr2, loss
            gc.collect()
            torch.cuda.empty_cache()
            print(f'\t\tloss = {loss_val:.4f}')

    gc.collect()
    torch.cuda.empty_cache()


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


iteration num: 0
	reprompt attempt 0
